In [1]:
from datasets import load_from_disk, load_dataset
from datasets import load_dataset, load_from_disk
import sys
import json
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor

from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.spice.spice import Spice
from pycocoevalcap.cider.cider import Cider
deplot_test = load_dataset('alexshengzhili/SciCapQA-test-with-deplot', split='1_percent_as_validation')
data = deplot_test.filter(lambda example: len(example['q_a_pairs']) > 0)



/home/ubuntu/mambaforge-pypy3/envs/llavav2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset parquet (/home/ubuntu/.cache/huggingface/datasets/alexshengzhili___parquet/alexshengzhili--SciCapQA-test-with-deplot-1ac41e44312d836e/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/alexshengzhili___parquet/alexshengzhili--SciCapQA-test-with-deplot-1ac41e44312d836e/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-78d2f229bcfd37cf.arrow


In [2]:
import json

# load this file /home/ubuntu/LLaVA-Graph/evaluation/llava-7b-v0/llava_answers_3k.jsonl
# into a list of dictionaries
json_file = '/home/ubuntu/LLaVA/data/3k_response.jsonl'
with open(json_file, 'r') as f:
    llava_answers_3k = [json.loads(line) for line in f]

In [3]:
llava_list  = [item['text'] for item in llava_answers_3k]

In [7]:
for i in range(3000):
    target_question_length = len(data[i]['q_a_pairs'][0][0])
    assert llava_answers_3k[i]['prompt'][-target_question_length:] == data[i]['q_a_pairs'][0][0]

In [8]:
llava = data.add_column('llava_response', llava_list)

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/alexshengzhili___parquet/alexshengzhili--SciCapQA-test-with-deplot-1ac41e44312d836e/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-043f1c3de8854caa.arrow


In [10]:
def evaluate_captions(gt_captions, pred_captions, print_scores=True):
    evaluators = [Bleu(4), Rouge(), Cider()]
    
    results = {}
    for evaluator in evaluators:
        score, _ = evaluator.compute_score(gt_captions, pred_captions)
        if isinstance(evaluator, Bleu):
            score = score[-1]  # Use the BLEU-4 score
        results[evaluator.method()] = score
    
    if print_scores:
        print("----- Evaluation Scores -----")
        for metric, score in results.items():
            print(f"{metric}: {score}")
    
    return results

def captions_list_to_dict(captions_list):
    captions_dict = {}
    for idx, caption in enumerate(captions_list):
        captions_dict[idx] = [caption]
    return captions_dict

gt_mplug = [example[0][1] for example in llava['q_a_pairs']]
pred_mplug = [example['llava_response'] for example in llava]
evaluate_captions(captions_list_to_dict(gt_mplug),
                    captions_list_to_dict(pred_mplug))

{'testlen': 243764, 'reflen': 189848, 'guess': [243764, 240762, 237760, 234758], 'correct': [75416, 30576, 16875, 10880]}
ratio: 1.2839956175466622
----- Evaluation Scores -----
Bleu: 0.10662278800245038
Rouge: 0.2737924751172802
CIDEr: 0.15378719095629606


{'Bleu': 0.10662278800245038,
 'Rouge': 0.2737924751172802,
 'CIDEr': 0.15378719095629606}

In [14]:
from huggingface_hub import HfApi
api = HfApi()
#api.create_repo('llava-eval-with-deplot', repo_type='dataset')
llava.push_to_hub('alexshengzhili/llava-eval-with-deplot')


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:00<00:00,  1.55it/s]
